# 01 Multi-Factor Regression with Controls (Micro)

Add controls, discuss omitted variable bias, robust SE.


## Table of Contents
- [Choose controls](#choose-controls)
- [Fit model](#fit-model)
- [Compare coefficients](#compare-coefficients)
- [Checkpoint (Self-Check)](#checkpoint-self-check)
- [Solutions (Reference)](#solutions-reference)


## Why This Notebook Matters
Regression is the bridge between statistics and ML. You will learn:
- single-factor vs multi-factor interpretation,
- robust standard errors,
- coefficient stability and multicollinearity.


## Prerequisites (Quick Self-Check)
- Completed Parts 00–01 (foundations + data).
- Basic algebra comfort (reading coefficient tables, units).

## What You Will Produce
- (no file output; learning/analysis notebook)

## Success Criteria
- You can explain what you built and why each step exists.
- You can run your work end-to-end without undefined variables.

## Common Pitfalls
- Running cells top-to-bottom without reading the instructions.
- Leaving `...` placeholders in code cells.
- Treating coefficients as causal without a causal design.
- Ignoring multicollinearity (unstable coefficients).

## Quick Fixes (When You Get Stuck)
- If you see `ModuleNotFoundError`, re-run the bootstrap cell and restart the kernel; make sure `PROJECT_ROOT` is the repo root.
- If a `data/processed/*` file is missing, either run the matching build script (see guide) or use the notebook’s `data/sample/*` fallback.
- If results look “too good,” suspect leakage; re-check shifts, rolling windows, and time splits.
- If a model errors, check dtypes (`astype(float)`) and missingness (`dropna()` on required columns).

## Matching Guide
- `docs/guides/02_regression/01_multifactor_regression_micro_controls.md`



## How To Use This Notebook
- Work section-by-section; don’t skip the markdown.
- Most code cells are incomplete on purpose: replace TODOs and `...`, then run.
- After each section, write 2–4 sentences answering the interpretation prompts (what changed, why it matters).
- Prefer `data/processed/*` if you have built the real datasets; otherwise use the bundled `data/sample/*` fallbacks.
- Use the **Checkpoint (Self-Check)** section to catch mistakes early.
- Use **Solutions (Reference)** only to unblock yourself; then re-implement without looking.
- Use the matching guide (`docs/guides/02_regression/01_multifactor_regression_micro_controls.md`) for the math, assumptions, and deeper context.



<a id="environment-bootstrap"></a>
## Environment Bootstrap
Run this cell first. It makes the repo importable and defines common directories.



In [ ]:
from __future__ import annotations

from pathlib import Path
import sys


def find_repo_root(start: Path) -> Path:
    p = start
    for _ in range(8):
        if (p / 'src').exists() and (p / 'docs').exists():
            return p
        p = p.parent
    raise RuntimeError('Could not find repo root. Start Jupyter from the repo root.')


PROJECT_ROOT = find_repo_root(Path.cwd())
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

DATA_DIR = PROJECT_ROOT / 'data'
RAW_DIR = DATA_DIR / 'raw'
PROCESSED_DIR = DATA_DIR / 'processed'
SAMPLE_DIR = DATA_DIR / 'sample'

PROJECT_ROOT



## Goal
Fit a multi-factor regression on county data, add controls, and discuss omitted variable bias (OVB).

Big idea:
- The coefficient on income can change when you add controls.
- That change is a clue that the simple model was absorbing other effects.



## Primer: `statsmodels` vs `scikit-learn` (inference vs prediction)

This repo uses both libraries because they serve different goals:

- **Prediction (ML):** optimize out-of-sample accuracy → `scikit-learn`
- **Inference (econometrics):** interpret coefficients + quantify uncertainty → `statsmodels`

### Minimal `statsmodels` OLS pattern

```python
import statsmodels.api as sm

# X: DataFrame of features, y: Series target
Xc = sm.add_constant(X, has_constant="add")  # add intercept
res = sm.OLS(y, Xc).fit()
print(res.summary())
```

**Expected output / sanity check**
- a table with `coef`, `std err`, `t`, `P>|t|`, and a CI column
- coefficient names match your column names

### What you are looking at in `res.summary()`

- **coef**: $\\hat\\beta$ (estimated effect in the model)
- **std err**: estimated uncertainty $\\widehat{SE}(\\hat\\beta)$
- **t**: $\\hat\\beta / \\widehat{SE}(\\hat\\beta)$
- **P>|t|**: p-value for $H_0: \\beta=0$ (conditional on assumptions)
- **[0.025, 0.975]**: 95% confidence interval

### Robust standard errors (change uncertainty, not coefficients)

```python
# Cross-section heteroskedasticity
res_hc3 = res.get_robustcov_results(cov_type="HC3")

# Time series autocorrelation + heteroskedasticity
res_hac = res.get_robustcov_results(cov_type="HAC", cov_kwds={"maxlags": 4})
```

### Common pitfalls (and quick fixes)

- **Forgetting the intercept**
  - Fix: always `add_constant`.
- **Wrong SE for time series**
  - Fix: use HAC when residuals are autocorrelated.
- **Treating p-values as causal proof**
  - Fix: write the identification assumption; otherwise interpret as association.
- **Mixing prediction and inference**
  - Fix: use `sklearn` pipelines + time splits for prediction; use `statsmodels` for coefficient uncertainty.


## Primer: Hypothesis testing (p-values, t-stats, confidence intervals)

You will see p-values, t-statistics, and confidence intervals in regression output (especially `statsmodels`). This primer gives you the minimum to interpret them correctly.

### The objects (plain language)

- **Null hypothesis** $H_0$: the default claim (often “no effect”).
- **Alternative** $H_1$: the claim you consider if the data looks inconsistent with $H_0$.
- **Test statistic**: “how far” your estimate is from the null, in uncertainty units.
- **p-value**: probability (under the null *and model assumptions*) of seeing a test statistic at least as extreme as observed.
- **Confidence interval (CI)**: a range of parameter values consistent with the data under assumptions.

### What a p-value is NOT

- Not the probability $H_0$ is true.
- Not the probability the model is correct.
- Not a measure of economic importance.

### Regression t-test intuition

In OLS, a common test is $H_0: \\beta_j = 0$.

$$
t_j = \\frac{\\hat\\beta_j}{\\widehat{SE}(\\hat\\beta_j)}
$$

If you change your SE estimator (HC3/HAC/cluster), you change $\\widehat{SE}$ and therefore the p-value, even if the coefficient stays the same.

### Expected output / what you should look at in `res.summary()`

- `coef`: effect size (in model units)
- `std err`: uncertainty
- CI columns: magnitude + uncertainty together

### Common pitfalls in this project

- Macro time series often have autocorrelation → naive SE too small → use HAC when interpreting p-values.
- Multiple testing/spec-search can produce small p-values by chance.
- Predictive success ≠ causal interpretation.

### Tiny demo (toy; not project data)

```python
import numpy as np
import pandas as pd
import statsmodels.api as sm

rng = np.random.default_rng(0)
n = 300
x = rng.normal(size=n)
y = 1.0 + 0.5 * x + rng.normal(scale=1.0, size=n)

df = pd.DataFrame({"y": y, "x": x})
X = sm.add_constant(df[["x"]])
res = sm.OLS(df["y"], X).fit()
print(res.summary())
```


<a id="choose-controls"></a>
## Choose controls

### Goal
Pick a set of plausible controls to include alongside income.

Starter controls (if you have them):
- `poverty_rate`
- `unemployment_rate`
- `log_population`
- `log_home_value` (if available)



### Your Turn (1): Load data and build baseline variables


In [ ]:
import numpy as np
import pandas as pd

year = 2022
path = PROCESSED_DIR / f'census_county_{year}.csv'
if path.exists():
    df = pd.read_csv(path)
else:
    df = pd.read_csv(SAMPLE_DIR / 'census_county_sample.csv')

# Outcome + main regressor
income = pd.to_numeric(df['B19013_001E'], errors='coerce')
rent = pd.to_numeric(df['B25064_001E'], errors='coerce')

# TODO: Build baseline log variables
mask = (income > 0) & (rent > 0)
df_m = pd.DataFrame({
    'log_income': np.log(income[mask]),
    'log_rent': np.log(rent[mask]),
}).dropna()

# TODO: Merge in control columns you want to use (from df)
# Hint: df_m = df_m.join(...)
...



### Your Turn (2): Choose controls list


In [ ]:
# TODO: Choose at least 2 controls available in df_m.
# Example: ['poverty_rate', 'unemployment_rate']
controls = [
    ...,
]

controls



### Note: Dummy variable encoding and the dummy variable trap

When you include a categorical variable (like state or region) in a regression, you must convert it to numeric dummy (indicator) columns. If your categorical has $k$ categories and you include all $k$ dummies plus an intercept, the columns are perfectly collinear (they sum to 1 = the intercept column). This is the **dummy variable trap**.

**Solution**: Drop one category (the "reference" or "base" category). Each remaining dummy coefficient is interpreted as the difference from the omitted category.

In pandas:
```python
# drop_first=True avoids the trap
dummies = pd.get_dummies(df['region'], prefix='region', drop_first=True)
df_m = df_m.join(dummies)
```

Alternatively, `statsmodels` `add_constant` handles this if you are careful, and `linearmodels` fixed effects absorb entity dummies automatically.

**Interpretation**: If you drop "Northeast" as the base, then `region_South = 0.05` means "counties in the South have 5 percentage points higher [outcome] than counties in the Northeast, holding other controls fixed."

<a id="fit-model"></a>
## Fit model

### Goal
Fit:
1) baseline model: log_rent ~ log_income
2) controlled model: log_rent ~ log_income + controls



### Your Turn (1): Fit baseline and controlled models (HC3)


In [ ]:
from src import econometrics

# Baseline
res_base = econometrics.fit_ols_hc3(df_m, y_col='log_rent', x_cols=['log_income'])

# Controlled
x_cols = ['log_income'] + controls
res_ctrl = econometrics.fit_ols_hc3(df_m, y_col='log_rent', x_cols=x_cols)

print(res_base.summary())
print(res_ctrl.summary())



### Optional: cluster-robust SE by state


In [ ]:
# Advanced (optional): if you have a 'state' column, research statsmodels cluster SE.
# The idea: errors may be correlated within a state.
# TODO: Try cov_type='cluster' and compare SE to HC3.
...



<a id="compare-coefficients"></a>
## Compare coefficients

### Goal
Compare the income coefficient with and without controls.

Interpretation prompt:
- If the coefficient changes a lot, what omitted factors might income have been proxying for?



### Your Turn (1): Build a comparison table


In [ ]:
import pandas as pd

def coef_row(res, name):
    return pd.Series({
        'coef': float(res.params[name]),
        'se': float(res.bse[name]),
        'p': float(res.pvalues[name]),
    })

comp = pd.DataFrame({
    'baseline': coef_row(res_base, 'log_income'),
    'controlled': coef_row(res_ctrl, 'log_income'),
}).T
comp



### Your Turn (2): Multicollinearity check (VIF)


In [ ]:
from src.econometrics import vif_table

# TODO: Compute VIF for the controlled model predictors (excluding the intercept).
vif = vif_table(df_m.dropna(), x_cols)
vif



<a id="checkpoint-self-check"></a>
## Checkpoint (Self-Check)
Run a few asserts and write 2-3 sentences summarizing what you verified.



In [ ]:
# TODO: Validate data and model results (cross-sectional data, no time splits needed).
# Example checks (adjust variable names):
# assert df_m.shape[0] > 100, "Too few observations after filtering"
# assert not df_m[x_cols].isna().any().any(), "NaNs in predictors"
# assert comp.shape[0] == 2, "Should have baseline + controlled row"
# assert all(vif['VIF'] > 0), "VIF values should be positive"
...

## Extensions (Optional)
- Try one additional variant beyond the main path (different features, different split, different model).
- Write down what improved, what got worse, and your hypothesis for why.



## Reflection
- What did you assume implicitly (about timing, availability, stationarity, or costs)?
- If you had to ship this model, what would you monitor?



<a id="solutions-reference"></a>
## Solutions (Reference)

Try the TODOs first. Use these only to unblock yourself or to compare approaches.

<details><summary>Solution: Choose controls</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 01_multifactor_regression_micro_controls — Choose controls
import numpy as np
import pandas as pd
from src import econometrics

df = pd.read_csv(SAMPLE_DIR / 'census_county_sample.csv')
df = df[(df['B19013_001E'] > 0) & (df['B25064_001E'] > 0) & (df['B01003_001E'] > 0)].copy()
df['log_income'] = np.log(df['B19013_001E'].astype(float))
df['log_rent'] = np.log(df['B25064_001E'].astype(float))
df['log_pop'] = np.log(df['B01003_001E'].astype(float))

res = econometrics.fit_ols_hc3(df, y_col='log_rent', x_cols=['log_income', 'log_pop', 'poverty_rate'])
print(res.summary())
```

</details>

<details><summary>Solution: Fit model</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 01_multifactor_regression_micro_controls — Fit model
# See above.
```

</details>

<details><summary>Solution: Compare coefficients</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 01_multifactor_regression_micro_controls — Compare coefficients
# Compare single-factor vs multi-factor slopes for log_income.
# If it changes a lot, omitted variable bias is plausible.
```

</details>

